# Lab 4
## 1606885864 - Ahmad Satryaji Aulia
### Initialization

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk

# Surpress warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
import warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action="ignore", category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # Do stuff here

In [2]:
# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

### Load Datasets

In [3]:
df = pd.read_csv("spam.csv")
df["Message"] = df["Message"].astype("str")
df["Mark"] = df["Mark"].astype("str")
dic = pd.read_csv("dictionary.csv")
norm = pd.read_csv("key_norm.csv")

In [4]:
df.dtypes

Name       object
Mark       object
Date       object
Message    object
dtype: object

In [5]:
df.head()

,Name,Mark,Date,Message
0,Z-SMS,Spam,7/4/2018 17:33,BAREKSA: Silakan tandatangani FPR di : https:/...
1,Z-SMS,Spam,7/4/2018 17:35,Terimakasih. Nomor ini telah terdaftar untuk v...
2,Z-SMS,Spam,7/9/2018 14:24,Mohon maaf transaksi pembelian/penjualan/penga...
3,Z-SMS,Spam,7/10/2018 13:21,Mohon maaf transaksi pembelian/penjualan/penga...
4,Z-SMS,Spam,8/7/2018 14:08,Mohon maaf transaksi reksadana Anda di Bareksa...


In [6]:
# Punctuation removal
def remove_punct(text):
    text = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text) # Remove numbers
    return text

df["Message"] = df["Message"].apply(lambda x: remove_punct(x))
df.head()

,Name,Mark,Date,Message
0,Z-SMS,Spam,7/4/2018 17:33,BAREKSA Silakan tandatangani FPR di httpsmbar...
1,Z-SMS,Spam,7/4/2018 17:35,Terimakasih Nomor ini telah terdaftar untuk ve...
2,Z-SMS,Spam,7/9/2018 14:24,Mohon maaf transaksi pembelianpenjualanpengali...
3,Z-SMS,Spam,7/10/2018 13:21,Mohon maaf transaksi pembelianpenjualanpengali...
4,Z-SMS,Spam,8/7/2018 14:08,Mohon maaf transaksi reksadana Anda di Bareksa...


In [7]:
# Kata tidak baku ke baku
norm_dic = pd.Series(norm.hasil.values,index=norm.singkat).to_dict()

def replace_norm(text):
    text_tokenized = nltk.word_tokenize(text.lower())
    text = " ".join(word if word not in norm_dic else norm_dic[word] for word in text_tokenized)
    return text

df["Message"] = df["Message"].apply(lambda x: replace_norm(x))
df.head()

,Name,Mark,Date,Message
0,Z-SMS,Spam,7/4/2018 17:33,bareksa silakan tandatangani fpr di httpsmbare...
1,Z-SMS,Spam,7/4/2018 17:35,terimakasih nomor ini telah terdaftar untuk ve...
2,Z-SMS,Spam,7/9/2018 14:24,mohon maaf transaksi pembelianpenjualanpengali...
3,Z-SMS,Spam,7/10/2018 13:21,mohon maaf transaksi pembelianpenjualanpengali...
4,Z-SMS,Spam,8/7/2018 14:08,mohon maaf transaksi reksadana anda di bareksa...


In [8]:
# Stemming
sf = StemmerFactory()
stemmer = sf.create_stemmer()
df["Message"] = df["Message"].map(stemmer.stem)
df.head()

,Name,Mark,Date,Message
0,Z-SMS,Spam,7/4/2018 17:33,bareksa sila tandatangani fpr di httpsmbareksa...
1,Z-SMS,Spam,7/4/2018 17:35,terimakasih nomor ini telah daftar untuk verif...
2,Z-SMS,Spam,7/9/2018 14:24,mohon maaf transaksi pembelianpenjualanpengali...
3,Z-SMS,Spam,7/10/2018 13:21,mohon maaf transaksi pembelianpenjualanpengali...
4,Z-SMS,Spam,8/7/2018 14:08,mohon maaf transaksi reksadana anda di bareksa...


### Wordcloud

### Feature Extraction
#### Bag of Words

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Vectorize
bow_transformer = CountVectorizer().fit(df["Message"])
# print total number of vocab words
print("Jumlah total vocabulary:",len(bow_transformer.vocabulary_))
# example of vectorized text
sample_tweet = df["Message"][50]
# Vector representation
bow_sample = bow_transformer.transform([sample_tweet])
print("Representasi vektor \"{}\"\n{}".format(sample_tweet, bow_sample))

Jumlah total vocabulary: 3318
Representasi vektor "promo panjang dapat potong rpbulan total cashback hingga juta hanya di xl prioritas daftar di xl center atau hubung mp"
  (0, 169)	1
  (0, 516)	1
  (0, 528)	1
  (0, 620)	1
  (0, 631)	1
  (0, 687)	2
  (0, 1024)	1
  (0, 1063)	1
  (0, 1192)	1
  (0, 1382)	1
  (0, 1844)	1
  (0, 2057)	1
  (0, 2179)	1
  (0, 2196)	1
  (0, 2208)	1
  (0, 2375)	1
  (0, 2899)	1
  (0, 3223)	2


In [10]:
messages_bow = bow_transformer.transform(df["Message"])
# Check out the bag-of-words counts for the entire corpus as a large sparse matrix
print('Bag of words sparse matrix shape: ', messages_bow.shape)
print('Non zero occurences: ', messages_bow.nnz)

Bag of words sparse matrix shape:  (3103, 3318)
Non zero occurences:  53431


#### Tf-Idf

In [11]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf_sample = tfidf_transformer.transform(bow_sample)
print(tfidf_sample)

  (0, 3223)	0.2549637872645276
  (0, 2899)	0.22066971885727155
  (0, 2375)	0.37757436929474997
  (0, 2208)	0.16805447545858374
  (0, 2196)	0.26409754695536053
  (0, 2179)	0.2762145291656409
  (0, 2057)	0.19544599732828602
  (0, 1844)	0.2955068012101702
  (0, 1382)	0.2518879297325356
  (0, 1192)	0.1508623494916039
  (0, 1063)	0.21387460351303852
  (0, 1024)	0.16697727140693125
  (0, 687)	0.19325514234134752
  (0, 631)	0.14986593678225058
  (0, 620)	0.24742045144432445
  (0, 528)	0.2612295572819805
  (0, 516)	0.2787559431371377
  (0, 169)	0.13087216786882247


### Classification

In [12]:
from sklearn.pipeline import Pipeline

# Create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', None), # do not set classifier yet
])

# Pipeline params for tuning
parameters = {'bow__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False)}

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Classifiers
clfs = {}
clfs["AdaBoost"] = AdaBoostClassifier()
clfs["Naive Bayes - Bernoulli"] = BernoulliNB()
clfs["Naive Bayes - Multinomial"] = MultinomialNB()
clfs["GradientBoosting"] = GradientBoostingClassifier()
clfs["RandomForest"] = RandomForestClassifier()
clfs["LogisticRegression"] = LogisticRegression()
clfs["SVC"] = SVC()

grid_searches = {}
# Do 10-fold cross validation for each model for each of the 4 possible param combinations
for clf in clfs:
    print("Running GridSearchCV for {}.".format(clf))
    pipeline.set_params(classifier = clfs[clf])
    grid = GridSearchCV(pipeline, cv=10, param_grid=parameters)
    grid.fit(df["Message"],df["Mark"])
    grid_searches[clf] = grid

Running GridSearchCV for AdaBoost.
Running GridSearchCV for Naive Bayes - Bernoulli.
Running GridSearchCV for Naive Bayes - Multinomial.
Running GridSearchCV for GradientBoosting.
Running GridSearchCV for RandomForest.
Running GridSearchCV for LogisticRegression.
Running GridSearchCV for SVC.


In [28]:
def score_summary(grid_searches, sort_by='mean_score'):
    def row(key, scores, params):
        d = {
             'estimator': key,
             'min_score': min(scores),
             'max_score': max(scores),
             'mean_score': np.mean(scores),
             'std_score': np.std(scores),
        }
        return pd.Series({**params,**d})

    rows = []
    for k in grid_searches:
        params = grid_searches[k].cv_results_['params']
        scores = []
        for i in range(grid_searches[k].cv):
            key = "split{}_test_score".format(i)
            r = grid_searches[k].cv_results_[key]        
            scores.append(r.reshape(len(params),1))

        all_scores = np.hstack(scores)
        for p, s in zip(params,all_scores):
            rows.append((row(k, s, p)))

    df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

    columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
    columns = columns + [c for c in df.columns if c not in columns]

    return df[columns]

score_summary(grid_searches).reset_index(drop=True)

,estimator,min_score,mean_score,max_score,std_score,bow__ngram_range,tfidf__use_idf
0,RandomForest,0.970874,0.985816,0.993569,0.00845128,"(1, 1)",False
1,RandomForest,0.97411,0.984528,0.990354,0.00573905,"(1, 1)",True
2,RandomForest,0.974277,0.984208,0.996785,0.00601236,"(1, 2)",True
3,RandomForest,0.96463,0.980014,0.987138,0.00688718,"(1, 2)",False
4,LogisticRegression,0.970968,0.976149,0.983923,0.00388496,"(1, 1)",False
5,GradientBoosting,0.951768,0.976144,0.990323,0.01156,"(1, 1)",False
6,GradientBoosting,0.954984,0.975825,0.993569,0.0120776,"(1, 1)",True
7,GradientBoosting,0.94822,0.971947,0.987138,0.0140992,"(1, 2)",False
8,LogisticRegression,0.967638,0.971314,0.977492,0.00307664,"(1, 2)",False
9,LogisticRegression,0.964516,0.970991,0.980707,0.00384204,"(1, 1)",True
